<a href="https://colab.research.google.com/github/MMillward2012/deepmind_internship/blob/main/notebooks/0_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔧 Project Setup

This notebook sets up the environment for the **Efficient and Interpretable Financial Sentiment Classification** project.

It installs dependencies, authenticates with Hugging Face, and confirms everything is ready to run model notebooks.

# Clone repo

In [ ]:
import os
if not os.path.exists('deepmind_internship'):
    !git clone https://github.com/MMillward2012/deepmind_internship.git

%cd deepmind_internship

# ✅ Install required packages

In [1]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


# ✅ Optional: Authenticate with Hugging Face (if accessing private models or uploading)

In [2]:
from huggingface_hub import notebook_login

notebook_login()


# ✅ Check GPU status

In [ ]:
import torch

if torch.backends.mps.is_available():
    print("✅ Apple MPS (Metal) GPU is available")
else:
    print("⚠️ No GPU available (using CPU)")


⚠️ GPU not available. You may want to switch to a GPU runtime.


# 📂 Folder structure check
These folders should already exist in the repo. If not, you can uncomment the lines below to create them manually.

In [3]:
# from pathlib import Path

# for folder in ["data", "outputs"]:
#     Path(folder).mkdir(exist_ok=True)


# ✅ Setup complete.
You can now run the model notebooks in `notebooks/`.